In [164]:
import pandas as pd
import numpy as np
product_data = pd.read_csv(r'\optimize_quantity.csv')
product_data.head()
product_data.iloc[:,:-1] = round(product_data.iloc[:,:-1]/1500,2)
product_data

,SKU_1_Inv,SKU_2_Inv,SKU_3_Inv,SKU_4_Inv,SKU_5_Inv,SKU_6_Inv,SKU_7_Inv,SKU_8_Inv,SKU_9_Inv,SKU_10_Inv,SKU_11_Inv,SKU_12_Inv,SKU_13_Inv,SKU_14_Inv,SKU_15_Inv,utility_pred
0,0.04,0.08,0.0,0.04,0.00,0.08,0.15,0.24,0.00,0.00,0.13,0.00,0.06,0.01,0.16,65.352790
1,0.16,0.01,0.0,0.00,0.19,0.04,0.00,0.19,0.05,0.06,0.20,0.01,0.00,0.00,0.08,65.171770
2,0.00,0.12,0.0,0.00,0.01,0.13,0.10,0.05,0.07,0.16,0.19,0.02,0.00,0.00,0.15,64.925255
3,0.03,0.07,0.0,0.00,0.18,0.06,0.09,0.12,0.10,0.00,0.05,0.00,0.00,0.13,0.17,64.553024
4,0.00,0.03,0.0,0.00,0.17,0.07,0.07,0.11,0.00,0.14,0.13,0.10,0.00,0.06,0.12,64.214680


In [165]:
import pyomo.environ as pyo
from pyomo.environ import *
from pyomo.opt import SolverFactory

import pyomo.kernel as pmo

In [166]:
model = pyo.ConcreteModel()

In [167]:
model.x  = Var(range(15) ,bounds = (30,600) , within = pyo.Integers)

In [168]:
model.x.pprint()

x : Size=15, Index=x_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      0 :    30 :  None :   600 : False :  True : Integers
      1 :    30 :  None :   600 : False :  True : Integers
      2 :    30 :  None :   600 : False :  True : Integers
      3 :    30 :  None :   600 : False :  True : Integers
      4 :    30 :  None :   600 : False :  True : Integers
      5 :    30 :  None :   600 : False :  True : Integers
      6 :    30 :  None :   600 : False :  True : Integers
      7 :    30 :  None :   600 : False :  True : Integers
      8 :    30 :  None :   600 : False :  True : Integers
      9 :    30 :  None :   600 : False :  True : Integers
     10 :    30 :  None :   600 : False :  True : Integers
     11 :    30 :  None :   600 : False :  True : Integers
     12 :    30 :  None :   600 : False :  True : Integers
     13 :    30 :  None :   600 : False :  True : Integers
     14 :    30 :  None :   600 : False :  True : Integers


In [169]:
index_non_zero = [0,1,3,5,6,7,10,12,13,14]# index where value is non zero
model.Obj = Objective( sense = maximize ,
                      expr = sum(model.x[i]*product_data.iloc[0,i] for i in (index_non_zero))
                      )

print("Objective Function",model.Obj.pprint())


Obj : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : maximize : 0.04*x[0] + 0.08*x[1] + 0.04*x[3] + 0.08*x[5] + 0.15*x[6] + 0.24*x[7] + 0.13*x[10] + 0.06*x[12] + 0.01*x[13] + 0.16*x[14]
Objective Function None


In [170]:
model.demand = pyo.ConstraintList()
index1 = 0


    
model.demand.add(expr = sum(model.x[i] for i in index_non_zero) == 1500)
    
    
print(model.demand.pprint())

demand : Size=1, Index=demand_index, Active=True
    Key : Lower  : Body                                                                    : Upper  : Active
      1 : 1500.0 : x[0] + x[1] + x[3] + x[5] + x[6] + x[7] + x[10] + x[12] + x[13] + x[14] : 1500.0 :   True
None


# 

In [171]:
from pyomo.util.infeasible import log_infeasible_constraints
solver = pyo.SolverFactory("glpk")
solution = solver.solve(model , tee =True)

log_infeasible_constraints(model , log_expression=True, log_variables=True)

if (solution.solver.status == SolverStatus.ok) and (solution.solver.termination_condition == TerminationCondition.optimal):
    print("Solution is feasible and optimal")
    #print("Objective function value = ", model.obj())
elif solution.solver.termination_condition == TerminationCondition.infeasible:
    print ("Failed to find solution.")
    raise SystemError("Failed to find solution.Please reassign to couriers")
else:
    # something else is wrong
    print(str(solution.solver))
    

GLPSOL--GLPK LP/MIP Solver 5.0
Parameter(s) specified in the command line:
 --write C:\Users\ROOPAG~1\AppData\Local\Temp\tmp0i_3r547.glpk.raw --wglp
 C:\Users\ROOPAG~1\AppData\Local\Temp\tmp_xdteqz3.glpk.glp --cpxlp C:\Users\ROOPAG~1\AppData\Local\Temp\tmp3tca0ihk.pyomo.lp
Reading problem data from 'C:\Users\ROOPAG~1\AppData\Local\Temp\tmp3tca0ihk.pyomo.lp'...
2 rows, 11 columns, 11 non-zeros
10 integer variables, none of which are binary
56 lines were read
Writing problem data to 'C:\Users\ROOPAG~1\AppData\Local\Temp\tmp_xdteqz3.glpk.glp'...
50 lines were written
GLPK Integer Optimizer 5.0
2 rows, 11 columns, 11 non-zeros
10 integer variables, none of which are binary
Preprocessing...
1 row, 10 columns, 10 non-zeros
10 integer variables, none of which are binary
Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 1
Solving LP relaxation...
GLPK Simplex Optimizer 5.0

In [173]:
df = pd.DataFrame()
dict1 = model.x.get_values()
df['optimize_product_quantity'] = list(dict1.values())
df

,optimize_product_quantity
0,30.0
1,30.0
2,NaN
3,30.0
4,NaN
5,30.0
6,90.0
7,600.0
8,NaN
9,NaN


In [181]:
product_price = {'SKU_1_Pr':                   [1.000000],
'SKU_2_Pr':                   [6.000000],
'SKU_3_Pr':                   [4.000000],
'SKU_4_Pr':                   [6.000000],
'SKU_5_Pr':                [6.000000],
'SKU_6_Pr':                  [2.000000],
'SKU_7_Pr':                   [5.000000],
'SKU_8_Pr':                   [6.000000],
'SKU_9_Pr':                  [5.000000],
'SKU_10_Pr':                 [5.000000],
'SKU_11_Pr':                  [2.000000],
'SKU_12_Pr':                [6.000000],
'SKU_13_Pr':               [1.000000],
'SKU_14_Pr':                  [2.000000],
'SKU_15_Pr':                 [5.000000]}
product_price = pd.DataFrame(product_price)
product_price = product_price.transpose()
product_price = product_price.reset_index()
product_price.columns = ['index','price']
product_price

,index,price
0,SKU_1_Pr,1.0
1,SKU_2_Pr,6.0
2,SKU_3_Pr,4.0
3,SKU_4_Pr,6.0
4,SKU_5_Pr,6.0
5,SKU_6_Pr,2.0
6,SKU_7_Pr,5.0
7,SKU_8_Pr,6.0
8,SKU_9_Pr,5.0
9,SKU_10_Pr,5.0


In [186]:
ans = (df['optimize_product_quantity'] * product_price['price'])
sum(ans[~ans.isna()])

7650.0